In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
#plt.style.use('fivethirtyeight')
plt.rcParams.update({'font.size': 12})
#plt.rcParams["figure.figsize"] = (12, 4)
from datetime import date

Install GluonTS

In [ ]:
!pip install -q gluonts
!pip install anvil-uplink

For our data we shall be using the dataset provided by the kaggle [G-Research Crypto Forecasting](https://www.kaggle.com/c/g-research-crypto-forecasting) competition. We shall extract the data corresponding to the Bitcoin cryptocurrency. From this Bitcoin data we shall use only one datapoint each day, arbitrarily chosen to be the **High** value at 9:00 a.m.

In [ ]:
dfs = []
train = pd.read_csv("../input/fuck-me/BTC.csv")
# extract the data corresponding to Bitcoin (Asset_ID = 1)
bitcoin = train.query("Asset_ID == 1").reset_index(drop = True)
bitcoin['timestamp'] = pd.to_datetime(bitcoin['timestamp'], unit='s')
bitcoin = bitcoin.set_index('timestamp')
# extract the "High" value at 9:00 a.m. daily
indexer_9am = bitcoin.index.indexer_at_time('9:00:00')
values_at_9am = bitcoin.iloc[indexer_9am]
high_values_at_9am = values_at_9am[["High"]]
# take a look
high_values_at_9am
bitcoin

We shall forecast three months (90 days) worth of data, based on the 180 days prior to the start of the forecasting period

In [ ]:
freq = "1D"             # the frequency of our data, here daily
context_length    = 2400 # train on this number of days
prediction_length = 140 # predict these many days, these are removed from the end of the training data

Convert our dataframe into a GluonTS dataset

In [ ]:
from gluonts.dataset.common import ListDataset

data_list = [{"start": "2017-01-01 09:00:00", "target": high_values_at_9am[c].values} for c in high_values_at_9am.columns]
train_ds  = ListDataset(data_iter=data_list,freq=freq)

In [ ]:
from gluonts.model.simple_feedforward  import SimpleFeedForwardEstimator
# to use the DeepAREstimator 
# from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.distribution.student_t import StudentTOutput
from gluonts.mx import Trainer

estimator = SimpleFeedForwardEstimator(num_hidden_dimensions=[15],
                                       freq=freq,
                                       context_length=context_length,
                                       prediction_length=prediction_length,
                                       distr_output=StudentTOutput(),
                                       trainer=Trainer(epochs=12,
                                                       learning_rate=1e-4,
                                                       num_batches_per_epoch=20,
                                                       patience=20))

predictor = estimator.train(train_ds)
print("Done")

In [ ]:
from gluonts.evaluation import make_evaluation_predictions

forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_ds,  # dataset
    predictor=predictor,  # predictor
    num_samples=2000,  # number of sample paths we want for evaluation
)

forecasts = list(forecast_it)
tss = list(ts_it)

In [ ]:
def plot_prob_forecasts(ts_entry, forecast_entry):
    
    from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
    from matplotlib.figure import Figure
    plot_length = context_length + prediction_length
    prediction_intervals = (50.0, 90.0)
    legend = ["ground truth", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

    fig, ax = plt.subplots(1, 1, figsize=(18, 7))
    ts_entry[-plot_length:].plot(ax=ax)  # plot the time series
    forecast_entry.plot(prediction_intervals=prediction_intervals, color='cadetblue')
    plt.axvline(forecast_entry.start_date, color='g', lw=1) # end of train dataset
    plt.grid(which="major")
    plt.legend(legend, loc="upper left")
    plt.show();
    plt.savefig('foo.png')
    return plt
    
plot_prob_forecasts(tss[0], forecasts[0])

